In [ ]:
import matplotlib
from matplotlib import pyplot as plt
from itertools import cycle
from pprint import pprint
import seaborn as sns
from math import floor
import pandas as pd
import numpy as np
import os

In [ ]:
def get_tofino_bandwidth(tf_version):
    if tf_version == 1:
        tf_bandwidth = 6.4 # Tbps
    elif tf_version == 2:
        tf_bandwidth = 12.8 # Tbps
    else:
        print("Error finding Tofino bandwidth")
    return tf_bandwidth

In [ ]:
def get_tofino_memory_limit(tf_version):

    MAX_MEMORY_PER_STAGE = 143500 # Found by compiling the Tofino program -- with different register array sizes -- on the hardware switch

    if tf_version == 1:
        memory_limit_slm = MAX_MEMORY_PER_STAGE * 8
        memory_limit_slr = MAX_MEMORY_PER_STAGE
    elif tf_version == 2:
        memory_limit_slm = MAX_MEMORY_PER_STAGE * 16
        memory_limit_slr = MAX_MEMORY_PER_STAGE * 3
    else:
        print("Error finding memory limits")

    return memory_limit_slm, memory_limit_slr

In [ ]:
def write_perf_data(tf_version, participants_range, meeting_count, scale_improvement):

    header  = ["ParticipantCount"]
    header += ["Software_OneSends", "Software_AllSend"]
    header += ["Bandwidth_OneSends", "Bandwidth_AllSend"]
    header += ["NRA_OneSends", "NRA_AllSend"]
    header += ["RA-R_OneSends", "RA-R_AllSend"]
    header += ["RA-SR_OneSends", "RA-SR_AllSend"]
    header += ["S-LM_OneSends", "S-LM_AllSend"]
    header += ["S-LR_OneSends", "S-LR_AllSend"]
    header += ["Hardware_NRA_OneSends", "Hardware_NRA_AllSend"]
    header += ["Hardware_RA_OneSends", "Hardware_RA_AllSend"]
    header += ["ScaleImprovement_NRA_Min", "ScaleImprovement_NRA_Max"]
    header += ["ScaleImprovement_RA_Min", "ScaleImprovement_RA_Max"]
    header += ["ScaleImprovement_Min", "ScaleImprovement_Max"]
    
    with open(f"data/tofino{tf_version}_data.csv", "w") as fp:
        fp.write(",".join(header) + "\n")
        lines = []

        for i, pm in enumerate(range(participants_range[0], participants_range[1]+1)):
            values = [ pm ]
            values += [ meeting_count["software_one"][i], meeting_count["software_all"][i] ]
            values += [ meeting_count["hardware_bw_one"][i], meeting_count["hardware_bw_all"][i] ]
            values += [ meeting_count["hardware_nra_one"][i], meeting_count["hardware_nra_all"][i] ]
            values += [ meeting_count["hardware_rar_one"][i], meeting_count["hardware_rar_all"][i] ]
            values += [ meeting_count["hardware_rasr_one"][i], meeting_count["hardware_rasr_all"][i] ]
            values += [ meeting_count["hardware_slm_one"][i], meeting_count["hardware_slm_all"][i] ]
            values += [ meeting_count["hardware_slr_one"][i], meeting_count["hardware_slr_all"][i] ]
            values += [ meeting_count["hardware_bottleneck_nra_one"][i], meeting_count["hardware_bottleneck_nra_all"][i] ]
            values += [ meeting_count["hardware_bottleneck_ra_one"][i], meeting_count["hardware_bottleneck_ra_all"][i] ]
            values += [ scale_improvement["min_nra"][i], scale_improvement["max_nra"][i] ]
            values += [ scale_improvement["min_ra"][i], scale_improvement["max_ra"][i] ]
            values += [ scale_improvement["min"][i], scale_improvement["max"][i] ]

            line = ",".join([str(v) for v in values])
            lines.append(line)

        fp.write("\n".join(lines))

In [ ]:
def compute_scallop_perf_for_combination(
        tf_version,
        num_participants, num_senders,
        rep_mode, sr_mode,
        num_quality_high, num_quality_mid, num_quality_low,
        retx_overhead
    ):

    if num_senders * (num_participants - 1) != num_quality_high + num_quality_mid + num_quality_low:
        print("Invalid distribution of qualities.")
    
    tf_bandwidth = get_tofino_bandwidth(tf_version)
    memory_limit_slm, memory_limit_slr = get_tofino_memory_limit(tf_version)
    MAX_REPLICATION_TREES = 64000 # 64K is the max. number of replication trees on the Tofino

    bw_per_stream_high = 3.0 # Mbps
    bw_per_stream_mid  = 1.5 # Mbps
    bw_per_stream_low  = 1.0 # Mbps

    # Software: maximum work
    sw_bandwidth = 32 * (8 * 10 * 10) * bw_per_stream_high # 32 cores

    # Work requirement per meeting
    ## Work requirement of the receivers
    sw_bw_req = (bw_per_stream_high * num_quality_high) + (bw_per_stream_mid * num_quality_mid) + (bw_per_stream_low * num_quality_low)
    ## Work requirement of the senders
    if num_quality_high > 0:
        sw_bw_req += bw_per_stream_high * num_senders
    elif num_quality_mid > 0:
        sw_bw_req += bw_per_stream_mid * num_senders
    else:
        sw_bw_req += bw_per_stream_low * num_senders
    ## Account for retransmission overhead
    sw_bw_req += (retx_overhead/100) * sw_bw_req

    ## Software: No. of meetings
    sw = int(sw_bandwidth // sw_bw_req)

    # Hardware
    
    ## Hardware bandwidth: requirement per meeting
    hw_bw_req = sw_bw_req
    ## Hardware: number of meetings
    hw_bw = int((tf_bandwidth * 10**6) // hw_bw_req)

    # Hardware: Replication
    hw_rep = None
    if num_participants > 2:
        if rep_mode == "NRA":
            hw_rep = MAX_REPLICATION_TREES * 2
        elif rep_mode == "RA-R":
            hw_rep = int(MAX_REPLICATION_TREES * 2 // 3)
        elif rep_mode == "RA-SR":
            hw_rep = int(MAX_REPLICATION_TREES * 2 // (3 * num_senders))

    # Hardware: Memory
    hw_mem = None
    if num_participants > 2:
        if sr_mode == "S-LM":
            hw_mem = int(memory_limit_slm // (num_senders * (num_participants - 1)))
        elif sr_mode == "S-LR":
            hw_mem = int(memory_limit_slr // (num_senders * (num_participants - 1)))

    # Hardware: Bottleneck
    hw = min(hw_x for hw_x in (hw_bw, hw_rep, hw_mem) if hw_x is not None)

    # Scale improvement
    scale = hw / sw

    # if tf_version == 2:
    #     print(tf_version, num_participants, num_senders,
    #         rep_mode, sr_mode,
    #         num_quality_high, num_quality_mid, num_quality_low,
    #         retx_overhead)
    #     print(sw, hw_bw, hw_rep, hw_mem, hw, scale)
    #     print()

    return sw, hw_bw, hw_rep, hw_mem, hw, scale

In [ ]:
compute_scallop_perf_for_combination(2, 10, 5, "RA-SR", "S-LM", 25, 15, 5, 1.0)

In [ ]:
def compute_scallop_perf(min_participants=2, max_participants=100):

    count = 0

    MIN_PARTICIPANTS = min_participants
    MAX_PARTICIPANTS = max_participants

    meeting_counts = {}
    for tf_version in ["tofino1", "tofino2"]:
        if tf_version not in meeting_counts:
            meeting_counts[tf_version] = {}
        for resource in ["software", "hardware_bw", "hardware_rep", "hardware_mem", "hardware"]:
            if resource not in meeting_counts[tf_version]:
                meeting_counts[tf_version][resource] = {}
            for metric in ["min", "max", "min_params", "max_params"]:
                if metric not in meeting_counts[tf_version][resource]:
                    meeting_counts[tf_version][resource][metric] = [None for _ in range(MIN_PARTICIPANTS, MAX_PARTICIPANTS + 1)]

    scale_improvements = {}
    for tf_version in ["tofino1", "tofino2"]:
        if tf_version not in scale_improvements:
            scale_improvements[tf_version] = {}
        for condition in ["non-rate-adapted", "rate-adapted", "all"]:
            if condition not in scale_improvements[tf_version]:
                scale_improvements[tf_version][condition] = {}
            for metric in ["min", "max", "min_params", "max_params"]:
                if metric not in scale_improvements[tf_version][condition]:
                    scale_improvements[tf_version][condition][metric] = [None for _ in range(MIN_PARTICIPANTS, MAX_PARTICIPANTS + 1)]

    # Tofino versions: 1, 2
    # Participants (M): 2 to 100
    # Senders per meeting (N): 1 to M
    # Replication modes: NRA, RA-R, RA-SR
    # Sequence number rewriting modes: S-LM, S-LR
    # High-quality streams (h): 0 to N(M-1)
    # Mid-quality streams (m): 0 to N(M-1) - h
    # Low-quality streams (l): N(M-1) - h - m
    # Retransmission overhead: 0 to 20% (intervals of 1%)
    
    for tf_v in [1, 2]:
        for n, num_participants in enumerate(range(MIN_PARTICIPANTS, MAX_PARTICIPANTS + 1)):
            params = []
            for num_senders in range(1, num_participants+1):
                num_receive_streams = num_senders * (num_participants - 1)
                for rep_mode in ["NRA", "RA-R", "RA-SR"]:
                    if rep_mode == "NRA":
                        params.append((num_senders, "NRA", None, num_receive_streams, 0, 0, 0))
                    else:
                        for sr_mode in ["S-LM", "S-LR"]:
                            for retx_overhead in range(21):
                                if rep_mode == "RA-R":
                                    # In RA-R mode, each receiver receives all its streams in the same quality
                                    for num_quality_high in range(0, num_receive_streams, num_senders):
                                        for num_quality_mid in range(0, num_receive_streams - num_quality_high + 1, num_senders):
                                            num_quality_low = num_receive_streams - num_quality_high - num_quality_mid
                                            params.append((num_senders, rep_mode, sr_mode, num_quality_high, num_quality_mid, num_quality_low, retx_overhead))
                                elif rep_mode == "RA-SR":
                                    for num_quality_high in range(num_receive_streams):
                                        for num_quality_mid in range(num_receive_streams - num_quality_high + 1):
                                            num_quality_low = num_receive_streams - num_quality_high - num_quality_mid
                                            # For RA-SR mode to be active, at least 2 quality levels should be active
                                            if sum(x > 0 for x in (num_quality_high, num_quality_mid, num_quality_low)) >= 2:
                                                params.append((num_senders, rep_mode, sr_mode, num_quality_high, num_quality_mid, num_quality_low, retx_overhead))
            
            for num_senders, rep_mode, sr_mode, num_quality_high, num_quality_mid, num_quality_low, retx_overhead in params:
                sw, hw_bw, hw_rep, hw_mem, hw, scale = compute_scallop_perf_for_combination (
                    tf_v, num_participants, num_senders, rep_mode, sr_mode, num_quality_high, num_quality_mid, num_quality_low, retx_overhead)

                params_ = (tf_v, num_participants, num_senders, rep_mode, sr_mode,
                     num_quality_high, num_quality_mid, num_quality_low, retx_overhead)
                if tf_v == 1:
                    tf_version = "tofino1"
                elif tf_v == 2:
                    tf_version = "tofino2"

                # Initialize
                ## Software
                if sw and not meeting_counts[tf_version]["software"]["min"][n]:
                    meeting_counts[tf_version]["software"]["min"][n] = sw
                    meeting_counts[tf_version]["software"]["min_params"][n] = params_
                if sw and not meeting_counts[tf_version]["software"]["max"][n]:
                    meeting_counts[tf_version]["software"]["max"][n] = sw
                    meeting_counts[tf_version]["software"]["max_params"][n] = params_
                ## Hardware bandwidth
                if hw_bw and not meeting_counts[tf_version]["hardware_bw"]["min"][n]:
                    meeting_counts[tf_version]["hardware_bw"]["min"][n] = hw_bw
                    meeting_counts[tf_version]["hardware_bw"]["min_params"][n] = params_
                if hw_bw and not meeting_counts[tf_version]["hardware_bw"]["max"][n]:
                    meeting_counts[tf_version]["hardware_bw"]["max"][n] = hw_bw
                    meeting_counts[tf_version]["hardware_bw"]["max_params"][n] = params_
                ## Hardware replication
                if hw_rep and not meeting_counts[tf_version]["hardware_rep"]["min"][n]:
                    meeting_counts[tf_version]["hardware_rep"]["min"][n] = hw_rep
                    meeting_counts[tf_version]["hardware_rep"]["min_params"][n] = params_
                if hw_rep and not meeting_counts[tf_version]["hardware_rep"]["max"][n]:
                    meeting_counts[tf_version]["hardware_rep"]["max"][n] = hw_rep
                    meeting_counts[tf_version]["hardware_rep"]["max_params"][n] = params_
                ## Hardware memory
                if hw_mem and not meeting_counts[tf_version]["hardware_mem"]["min"][n]:
                    meeting_counts[tf_version]["hardware_mem"]["min"][n] = hw_mem
                    meeting_counts[tf_version]["hardware_mem"]["min_params"][n] = params_
                if hw_mem and not meeting_counts[tf_version]["hardware_mem"]["max"][n]:
                    meeting_counts[tf_version]["hardware_mem"]["max"][n] = hw_mem
                    meeting_counts[tf_version]["hardware_mem"]["max_params"][n] = params_
                ## Hardware bottleneck
                if hw and not meeting_counts[tf_version]["hardware"]["min"][n]:
                    meeting_counts[tf_version]["hardware"]["min"][n] = hw
                    meeting_counts[tf_version]["hardware"]["min_params"][n] = params_
                if hw and not meeting_counts[tf_version]["hardware"]["max"][n]:
                    meeting_counts[tf_version]["hardware"]["max"][n] = hw
                    meeting_counts[tf_version]["hardware"]["max_params"][n] = params_
                ## Scale improvement
                for metric in ["min", "max"]:
                    if scale and not scale_improvements[tf_version]["all"][metric][n]:
                        scale_improvements[tf_version]["all"][metric][n] = scale
                for metric in ["min_params", "max_params"]:
                    if scale and not scale_improvements[tf_version]["all"][metric][n]:
                        scale_improvements[tf_version]["all"][metric][n] = (params_, sw, hw_bw, hw_rep, hw_mem, hw)
                if rep_mode == "NRA":
                    for metric in ["min", "max"]:
                        if scale and not scale_improvements[tf_version]["non-rate-adapted"][metric][n]:
                            scale_improvements[tf_version]["non-rate-adapted"][metric][n] = scale
                    for metric in ["min_params", "max_params"]:
                        if scale and not scale_improvements[tf_version]["non-rate-adapted"][metric][n]:
                            scale_improvements[tf_version]["non-rate-adapted"][metric][n] = (params_, sw, hw_bw, hw_rep, hw_mem, hw)
                elif rep_mode in ["RA-R", "RA-SR"]:
                    for metric in ["min", "max"]:
                        if scale and not scale_improvements[tf_version]["rate-adapted"][metric][n]:
                            scale_improvements[tf_version]["rate-adapted"][metric][n] = scale
                    for metric in ["min_params", "max_params"]:
                        if scale and not scale_improvements[tf_version]["rate-adapted"][metric][n]:
                            scale_improvements[tf_version]["rate-adapted"][metric][n] = (params_, sw, hw_bw, hw_rep, hw_mem, hw)
                
                # Adjustments
                ## Software
                if sw < meeting_counts[tf_version]["software"]["min"][n]:
                    meeting_counts[tf_version]["software"]["min"][n] = sw
                    meeting_counts[tf_version]["software"]["min_params"][n] = params_
                if sw > meeting_counts[tf_version]["software"]["max"][n]:
                    meeting_counts[tf_version]["software"]["max"][n] = sw
                    meeting_counts[tf_version]["software"]["max_params"][n] = params_
                ## Hardware bandwidth
                if hw_bw < meeting_counts[tf_version]["hardware_bw"]["min"][n]:
                    meeting_counts[tf_version]["hardware_bw"]["min"][n] = hw_bw
                    meeting_counts[tf_version]["hardware_bw"]["min_params"][n] = params_
                if hw_bw > meeting_counts[tf_version]["hardware_bw"]["max"][n]:
                    meeting_counts[tf_version]["hardware_bw"]["max"][n] = hw_bw
                    meeting_counts[tf_version]["hardware_bw"]["max_params"][n] = params_
                ## Hardware replication
                if hw_rep and meeting_counts[tf_version]["hardware_rep"]["min"][n] and hw_rep < meeting_counts[tf_version]["hardware_rep"]["min"][n]:
                    meeting_counts[tf_version]["hardware_rep"]["min"][n] = hw_rep
                    meeting_counts[tf_version]["hardware_rep"]["min_params"][n] = params_
                if hw_rep and meeting_counts[tf_version]["hardware_rep"]["max"][n] and hw_rep > meeting_counts[tf_version]["hardware_rep"]["max"][n]:
                    meeting_counts[tf_version]["hardware_rep"]["max"][n] = hw_rep
                    meeting_counts[tf_version]["hardware_rep"]["max_params"][n] = params_
                ## Hardware memory
                if hw_mem and meeting_counts[tf_version]["hardware_mem"]["min"][n] and hw_mem < meeting_counts[tf_version]["hardware_mem"]["min"][n]:
                    meeting_counts[tf_version]["hardware_mem"]["min"][n] = hw_mem
                    meeting_counts[tf_version]["hardware_mem"]["min_params"][n] = params_
                if hw_mem and meeting_counts[tf_version]["hardware_mem"]["max"][n] and hw_mem > meeting_counts[tf_version]["hardware_mem"]["max"][n]:
                    meeting_counts[tf_version]["hardware_mem"]["max"][n] = hw_mem
                    meeting_counts[tf_version]["hardware_mem"]["max_params"][n] = params_
                ## Hardware bottleneck
                if hw < meeting_counts[tf_version]["hardware"]["min"][n]:
                    meeting_counts[tf_version]["hardware"]["min"][n] = hw
                    meeting_counts[tf_version]["hardware"]["min_params"][n] = params_
                if hw > meeting_counts[tf_version]["hardware"]["max"][n]:
                    meeting_counts[tf_version]["hardware"]["max"][n] = hw
                    meeting_counts[tf_version]["hardware"]["max_params"][n] = params_
                ## Scale improvement
                if scale < scale_improvements[tf_version]["all"]["min"][n]:
                    scale_improvements[tf_version]["all"]["min"][n] = scale
                    scale_improvements[tf_version]["all"]["min_params"][n] = (params_, sw, hw_bw, hw_rep, hw_mem, hw)
                if scale > scale_improvements[tf_version]["all"]["max"][n]:
                    scale_improvements[tf_version]["all"]["max"][n] = scale
                    scale_improvements[tf_version]["all"]["max_params"][n] = (params_, sw, hw_bw, hw_rep, hw_mem, hw)
                if rep_mode == "NRA":
                    if scale < scale_improvements[tf_version]["non-rate-adapted"]["min"][n]:
                        scale_improvements[tf_version]["non-rate-adapted"]["min"][n] = scale
                        scale_improvements[tf_version]["non-rate-adapted"]["min_params"][n] = (params_, sw, hw_bw, hw_rep, hw_mem, hw)
                    if scale > scale_improvements[tf_version]["non-rate-adapted"]["max"][n]:
                        scale_improvements[tf_version]["non-rate-adapted"]["max"][n] = scale
                        scale_improvements[tf_version]["non-rate-adapted"]["max_params"][n] = (params_, sw, hw_bw, hw_rep, hw_mem, hw)
                elif rep_mode in ["RA-R", "RA-SR"]:
                    if scale < scale_improvements[tf_version]["rate-adapted"]["min"][n]:
                        scale_improvements[tf_version]["rate-adapted"]["min"][n] = scale
                        scale_improvements[tf_version]["rate-adapted"]["min_params"][n] = (params_, sw, hw_bw, hw_rep, hw_mem, hw)
                    if scale > scale_improvements[tf_version]["rate-adapted"]["max"][n]:
                        scale_improvements[tf_version]["rate-adapted"]["max"][n] = scale
                        scale_improvements[tf_version]["rate-adapted"]["max_params"][n] = (params_, sw, hw_bw, hw_rep, hw_mem, hw)
                
                count += 1
                if count%10**8 == 0: #100M
                    print(f"Version: {tf_version}; Interm count: {count//10**9}B")
                if count == 1000000:
                    break
            if count == 1000000:
                break
        if count == 1000000:
            break

    print(f"Final count: {count}")
    return meeting_counts, scale_improvements

In [ ]:
meeting_counts, scale_improvements = compute_scallop_perf(2, 3)

In [ ]:
pprint(meeting_counts["tofino2"])
print()
pprint(scale_improvements["tofino2"])
print()

In [ ]:
def generate_scallop_perf_data(tf_versions):

    MIN_PARTICIPANTS = 2
    MAX_PARTICIPANTS = 100
    MAX_REPLICATION_TREES = 64000 # 64K is the max. number of replication trees on the Tofino
    BANDWIDTH_CUT_RATE = 0.05 # Assuming 5% cut due to loss/retransmission in all cases,
    # although, in reality, the cut would be higher for S-LM vs. S-LR (5% vs. 2%, for example)

    for tf_version in tf_versions:
        print(f"Tofino version: {tf_version}")
        
        meeting_count = {
            "software_one": [], "software_all": [],
            "hardware_bw_one": [], "hardware_bw_all": [],
            "hardware_nra_one": [], "hardware_nra_all": [],
            "hardware_rar_one": [], "hardware_rar_all": [],
            "hardware_rasr_one": [], "hardware_rasr_all": [],
            "hardware_slm_one": [], "hardware_slm_all": [],
            "hardware_slr_one": [], "hardware_slr_all": [],
            "hardware_bottleneck_nra_one": [], "hardware_bottleneck_nra_all": [],
            "hardware_bottleneck_ra_one": [], "hardware_bottleneck_ra_all": []
        }
    
        scale_improvement = {
            "min_nra": [], "max_nra": [],
            "min_ra": [], "max_ra": [],
            "min": [], "max": []
        }

        tf_bandwidth = get_tofino_bandwidth(tf_version)
        memory_limit_slm, memory_limit_slr = get_tofino_memory_limit(tf_version)

        for pm in list(range(MIN_PARTICIPANTS, MAX_PARTICIPANTS+1)):
            # pm = number of participants per meetings
            # one = one participant sends, all = all participants send

            # 1. Software
            sw_one = 32 * (8 * 10 * 10) // pm # 32 cores x 800 receive streams
            sw_all  = 32 * (8 * 10 * 10) // (pm * pm) # 32 cores x 800 receive streams

            meeting_count["software_one"].append(sw_one)
            meeting_count["software_all"].append(sw_all)

            # 2. Hardware

            # 2.1. Bandwidth
            bw_one = int(floor((1 - BANDWIDTH_CUT_RATE) * (tf_bandwidth * 10**6) / (3 * pm)))
            bw_all = int(floor((1 - BANDWIDTH_CUT_RATE) * (tf_bandwidth * 10**6) / (3 * pm * pm)))

            meeting_count["hardware_bw_one"].append(bw_one)
            meeting_count["hardware_bw_all"].append(bw_all)

            # 2.2. Replication
            if pm == 2:
                # No replication for meetings with only 2 participants
                hw_nra_one = hw_nra_all = hw_rar_one = hw_rar_all = -1
                hw_rasr_one = hw_rasr_all = -1

            else:
                # 2.2.1. NRA
                hw_nra_one = MAX_REPLICATION_TREES * 2
                hw_nra_all = MAX_REPLICATION_TREES * 2

                # 2.2.2. RA-R
                hw_rar_one = MAX_REPLICATION_TREES * 2 // 3
                hw_rar_all = MAX_REPLICATION_TREES * 2 // 3

                # 2.2.3. RA-SR
                hw_rasr_one = MAX_REPLICATION_TREES * 2 // 3
                hw_rasr_all = MAX_REPLICATION_TREES * 2 // (3 * pm)

            meeting_count["hardware_nra_one"].append(hw_nra_one)
            meeting_count["hardware_nra_all"].append(hw_nra_all)

            meeting_count["hardware_rar_one"].append(hw_rar_one)
            meeting_count["hardware_rar_all"].append(hw_rar_all)

            meeting_count["hardware_rasr_one"].append(hw_rasr_one)
            meeting_count["hardware_rasr_all"].append(hw_rasr_all)

            # 2.3. Sequence number rewriting
            if pm == 2:
                # No sequence number rewriting for meetings with only 2 participants
                hw_slm_one = hw_slm_all = -1
                hw_slr_one = hw_slr_all = -1
                
            else:
                # 2.3.1. S-LM
                hw_slm_one = memory_limit_slm // (pm - 1)
                hw_slm_all = memory_limit_slm // (pm * (pm - 1))

                # 2.3.2. S-LR
                hw_slr_one = memory_limit_slr // (pm - 1)
                hw_slr_all = memory_limit_slr // (pm * (pm - 1))

            meeting_count["hardware_slm_one"].append(hw_slm_one)
            meeting_count["hardware_slm_all"].append(hw_slm_all)

            meeting_count["hardware_slr_one"].append(hw_slr_one)
            meeting_count["hardware_slr_all"].append(hw_slr_all)

            # 2.4. Hardware bottleneck
            if pm == 2:
                # The bottleneck is always the Tofino bandwidth when there are only 2 participants
                hw_bottleneck_nra_one = int(bw_one)
                hw_bottleneck_nra_all = int(bw_all)
                hw_bottleneck_ra_one  = int(bw_one)
                hw_bottleneck_ra_all  = int(bw_all)

            else:
                hw_bottleneck_nra_one = min(hw_nra_one, bw_one)
                hw_bottleneck_nra_all = min(hw_nra_all, bw_all)
                hw_bottleneck_ra_one  = min(hw_nra_one, hw_rar_one, hw_rasr_one, hw_slm_one, hw_slr_one, bw_one)
                hw_bottleneck_ra_all  = min(hw_nra_all, hw_rar_all, hw_rasr_all, hw_slm_all, hw_slr_all, bw_all)

            meeting_count["hardware_bottleneck_nra_one"].append(hw_bottleneck_nra_one)
            meeting_count["hardware_bottleneck_nra_all"].append(hw_bottleneck_nra_all)
            meeting_count["hardware_bottleneck_ra_one"].append(hw_bottleneck_ra_one)
            meeting_count["hardware_bottleneck_ra_all"].append(hw_bottleneck_ra_all)

            # 2.5. Scale improvement of hardware on software
            if pm == 2:
                scale_improvement_nra_min = min(hw_bottleneck_nra_one/sw_one, hw_bottleneck_nra_all/sw_all)
                scale_improvement_nra_max = max(bw_one/sw_one, bw_all/sw_all)
                scale_improvement_ra_min  = min(hw_bottleneck_ra_one/sw_one, hw_bottleneck_ra_all/sw_all)
                scale_improvement_ra_max  = max(bw_one/sw_one, bw_all/sw_all)
            else:
                scale_improvement_nra_min = min(hw_bottleneck_nra_one/sw_one, hw_bottleneck_nra_all/sw_all)
                scale_improvement_nra_max = max(min(bw_one, hw_nra_one)/sw_one, min(bw_all, hw_nra_all)/sw_all)
                scale_improvement_ra_min  = min(hw_bottleneck_ra_one/sw_one, hw_bottleneck_ra_all/sw_all)
                scale_improvement_ra_max  = max(min(bw_one, hw_nra_one)/sw_one, min(bw_all, hw_nra_all)/sw_all)
            
            scale_improvement["min"].append(scale_improvement_min)
            scale_improvement["max"].append(scale_improvement_max)

        write_perf_data(tf_version, (MIN_PARTICIPANTS, MAX_PARTICIPANTS), meeting_count, scale_improvement)

In [ ]:
TOFINO_VERSIONS = [1, 2]

In [ ]:
generate_scallop_perf_data(tf_versions=TF_VERSIONS)

## Parse the data

In [ ]:
meeting_counts = { 1: {}, 2: {} }
scale_improvements = { 1: {}, 2: {} }

for tf_version in meeting_counts:

    meeting_counts[tf_version] = {
        "software_one": [], "software_all": [],
        "hardware_bw_one": [], "hardware_bw_all": [],
        "hardware_nra_one": [], "hardware_nra_all": [],
        "hardware_rar_one": [], "hardware_rar_all": [],
        "hardware_rasr_one": [], "hardware_rasr_all": [],
        "hardware_slm_one": [], "hardware_slm_all": [],
        "hardware_slr_one": [], "hardware_slr_all": [],
        "hardware_bottleneck_one": [], "hardware_bottleneck_all": []
    }

    scale_improvements[tf_version] = {
        "min": [], "max": []
    }

In [ ]:
def load_valid_int_data(df, col):
    l = df[col].tolist()
    l_valid = [int(x) for x in l if int(x) > -1]
    return l_valid

In [ ]:
def load_valid_float_data(df, col):
    l = df[col].tolist()
    l_valid = [float(x) for x in l if float(x) > -1]
    return l_valid

In [ ]:
for tf_version in TOFINO_VERSIONS:
    df = pd.read_csv(f"data/tofino{tf_version}_data.csv")
    participant_count = load_valid_data(df, "ParticipantCount")
    meeting_counts[tf_version]["software_one"] = load_valid_int_data(df, "Software_OneSends")
    meeting_counts[tf_version]["software_all"] = load_valid_int_data(df, "Software_AllSend")
    meeting_counts[tf_version]["hardware_bw_one"] = load_valid_int_data(df, "Bandwidth_OneSends")
    meeting_counts[tf_version]["hardware_bw_all"] = load_valid_int_data(df, "Bandwidth_AllSend")
    meeting_counts[tf_version]["hardware_nra_one"] = load_valid_int_data(df, "NRA_OneSends")
    meeting_counts[tf_version]["hardware_nra_all"] = load_valid_int_data(df, "NRA_AllSend")
    meeting_counts[tf_version]["hardware_rar_one"] = load_valid_int_data(df, "RA-R_OneSends")
    meeting_counts[tf_version]["hardware_rar_all"] = load_valid_int_data(df, "RA-R_AllSend")
    meeting_counts[tf_version]["hardware_rasr_one"] = load_valid_int_data(df, "RA-SR_OneSends")
    meeting_counts[tf_version]["hardware_rasr_all"] = load_valid_int_data(df, "RA-SR_AllSend")
    meeting_counts[tf_version]["hardware_slm_one"] = load_valid_int_data(df, "S-LM_OneSends")
    meeting_counts[tf_version]["hardware_slm_all"] = load_valid_int_data(df, "S-LM_AllSend")
    meeting_counts[tf_version]["hardware_slr_one"] = load_valid_int_data(df, "S-LR_OneSends")
    meeting_counts[tf_version]["hardware_slr_all"] = load_valid_int_data(df, "S-LR_AllSend")
    meeting_counts[tf_version]["hardware_bottleneck_one"] = load_valid_int_data(df, "Hardware_OneSends")
    meeting_counts[tf_version]["hardware_bottleneck_all"] = load_valid_int_data(df, "Hardware_AllSend")
    scale_improvements[tf_version]["min"] = load_valid_float_data(df, "ScaleImprovement_Min")
    scale_improvements[tf_version]["max"] = load_valid_float_data(df, "ScaleImprovement_Max")

## Plot the data

In [ ]:
sns.set()
sns.set_style("whitegrid")
font = {'family' : 'serif',
        'size'   : 25}
matplotlib.rc('font', **font)
plt.rc('xtick',labelsize=23)
plt.rc('ytick',labelsize=25)
plt.rc('axes',labelsize=30)
plt.rc('axes',titlesize=30)
plt.rc('legend',fontsize=23)
colors = list(sns.color_palette("bright"))

### All parameters

In [ ]:
for tf_version in TOFINO_VERSIONS:

    # Plotting the lines
    plt.figure(figsize=(15, 6))
    
    plt.plot(participant_count, meeting_counts[tf_version]["software_one"], color=colors[0], label="Software (1 sender)", marker='o')
    plt.plot(participant_count, meeting_counts[tf_version]["software_all"], color=colors[0], label="Software (N senders)", marker='s')
    plt.fill_between(participant_count, meeting_counts[tf_version]["software_one"], meeting_counts[tf_version]["software_all"],
                     color=colors[0], alpha=0.3, label="Software range")

    plt.plot(participant_count, meeting_counts[tf_version]["hardware_bw_one"], color=colors[1], label="Hardware bandwidth (1 sender)", marker='o')
    plt.plot(participant_count, meeting_counts[tf_version]["hardware_bw_all"], color=colors[1], label="Hardware bandwidth (N senders)", marker='s')
    plt.fill_between(participant_count, meeting_counts[tf_version]["hardware_bw_one"], meeting_counts[tf_version]["hardware_bw_all"],
                     color=colors[1], alpha=0.3, label="Hardware bandwidth range")
    
    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_nra_one"], color=colors[2], label="Hardware NRA (1 sender)", marker='o')
    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_nra_all"], color=colors[2], label="Hardware NRA (N senders)", marker='s')
    plt.fill_between(participant_count[1:], meeting_counts[tf_version]["hardware_nra_one"], meeting_counts[tf_version]["hardware_nra_all"],
                     color=colors[2], alpha=0.3, label="Hardware NRA")
    
    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_rar_one"], color=colors[3], label="Hardware RA-R (1 sender)", marker='o')
    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_rar_all"], color=colors[3], label="Hardware RA-R (N senders)", marker='s')
    plt.fill_between(participant_count[1:], meeting_counts[tf_version]["hardware_rar_one"], meeting_counts[tf_version]["hardware_rar_all"],
                     color=colors[3], alpha=0.3, label="Hardware RA-R")
    
    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_rasr_one"], color=colors[4], label="Hardware RA-SR (1 sender)", marker='o')
    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_rasr_all"], color=colors[4], label="Hardware RA-SR (N senders)", marker='s')
    plt.fill_between(participant_count[1:], meeting_counts[tf_version]["hardware_rasr_one"], meeting_counts[tf_version]["hardware_rasr_all"],
                     color=colors[4], alpha=0.3, label="Hardware RA-SR")
    
    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_slm_one"], color=colors[5], label="Hardware S-LM (1 sender)", marker='o')
    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_slm_all"], color=colors[5], label="Hardware S-LM (N senders)", marker='s')
    plt.fill_between(participant_count[1:], meeting_counts[tf_version]["hardware_slm_one"], meeting_counts[tf_version]["hardware_slm_all"],
                     color=colors[5], alpha=0.3, label="Hardware S-LM")

    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_slr_one"], color=colors[6], label="Hardware S-LR (1 sender)", marker='o')
    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_slr_all"], color=colors[6], label="Hardware S-LR (N senders)", marker='s')
    plt.fill_between(participant_count[1:], meeting_counts[tf_version]["hardware_slr_one"], meeting_counts[tf_version]["hardware_slr_all"],
                     color=colors[6], alpha=0.3, label="Hardware S-LR")

    plt.plot(participant_count, meeting_counts[tf_version]["hardware_bottleneck_one"], color=colors[7], label="Hardware bottleneck (1 sender)", marker='o')
    plt.plot(participant_count, meeting_counts[tf_version]["hardware_bottleneck_all"], color=colors[7], label="Hardware bottleneck (N senders)", marker='s')
    plt.fill_between(participant_count, meeting_counts[tf_version]["hardware_bottleneck_one"], meeting_counts[tf_version]["hardware_bottleneck_all"],
                     color=colors[7], alpha=0.3, label="Hardware bottleneck")
    
    # Adding labels and legend
    plt.rc('legend',fontsize=10)
    plt.xlabel("Participant count per meeting")
    plt.ylabel("Meeting count")
    plt.yscale("log")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', ncol=2, borderaxespad=0.)
    plt.title(f"Tofino{tf_version}")
    plt.tight_layout()
    plt.savefig(f"plots/tofino{tf_version}_allparams_perf.pdf", dpi=300)
    plt.show()

## Bottlenecks

In [ ]:
for tf_version in TOFINO_VERSIONS:

    # Plotting the lines
    plt.figure(figsize=(11, 6))
    
    plt.plot(participant_count, meeting_counts[tf_version]["software_all"], color=colors[0], label="Software", lw=5, drawstyle='steps-post')
    plt.plot(participant_count, meeting_counts[tf_version]["hardware_bw_all"], color=colors[1], label="Bandwidth", lw=5)
    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_nra_all"], color=colors[2], label="NRA", lw=5, linestyle="-")
    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_rar_all"], color=colors[2], label="RA-R", lw=5, linestyle="--")
    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_rasr_all"], color=colors[2], label="RA-SR", lw=5, linestyle=":")
    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_slm_all"], color=colors[3], label="S-LM", lw=5, linestyle="-")
    plt.plot(participant_count[1:], meeting_counts[tf_version]["hardware_slr_all"], color=colors[3], label="S-LR", lw=5, linestyle="--")
    
    # Adding labels and legend
    plt.rc('legend',fontsize=25)
    plt.xlabel("Participant count per meeting")
    plt.ylabel("Meeting count")
    plt.yscale("log")
    plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
    plt.title(f"Tofino{tf_version}")
    plt.tight_layout()
    plt.savefig(f"plots/tofino_bottlenecks_tofino{tf_version}.pdf", dpi=300)
    plt.show()

In [ ]:
for tf_version in TOFINO_VERSIONS:
    
    # Plotting the lines
    plt.figure(figsize=(9, 6))
    
    plt.plot(participant_count, meeting_counts[tf_version]["software_one"], color=colors[0])
    plt.plot(participant_count, meeting_counts[tf_version]["software_all"], color=colors[0], drawstyle='steps-post')
    plt.fill_between(participant_count, meeting_counts[tf_version]["software_one"], meeting_counts[tf_version]["software_all"],
                     color=colors[0], alpha=0.3, label="Software")
    
    plt.plot(participant_count, meeting_counts[tf_version]["hardware_bottleneck_one"], color=colors[3])
    plt.plot(participant_count, meeting_counts[tf_version]["hardware_bottleneck_all"], color=colors[3])
    plt.fill_between(participant_count, meeting_counts[tf_version]["hardware_bottleneck_one"], meeting_counts[tf_version]["hardware_bottleneck_all"],
                     color=colors[3], alpha=0.3, label="Scallop")
    
    # Adding labels and legend
    plt.xlabel("Participant count per meeting")
    plt.ylabel("Meeting count")
    plt.yscale("log")
    plt.legend()
    plt.tight_layout()
    plt.title(f"Tofino{tf_version}")
    plt.savefig(f"plots/tofino{tf_version}_perf_minmax.pdf", dpi=300)
    plt.show()

In [ ]:
for tf_version in TOFINO_VERSIONS:

    # Plotting the lines
    plt.figure(figsize=(9, 6))
    
    plt.plot(participant_count, scale_improvements[tf_version]["min"], color=colors[4])
    plt.plot(participant_count, scale_improvements[tf_version]["max"], color=colors[4])
    plt.fill_between(
        participant_count, scale_improvements[tf_version]["min"], scale_improvements[tf_version]["max"],
        color=colors[4], alpha=0.3, label="Improvement range")
    
    # Adding labels and legend
    plt.xlabel("Participant count per meeting")
    plt.ylabel("Improvement over\nsoftware (times)")
    plt.tight_layout()
    plt.savefig(f"plots/tofino{tf_version}_scale_improvement.pdf", dpi=300)
    plt.show()

In [ ]:
for tf_version in TOFINO_VERSIONS:
    print(f"Tofino{tf_version}:")

    print(f"Min. scalability improvement: {min(scale_improvements[tf_version]["min"])}")
    print(f"Max. scalability improvement: {max(scale_improvements[tf_version]["max"])}")

## Plot performance data based on simulations in C++

In [ ]:
tofino1_perfdata_path = "data/tofino1.csv"
tofino2_perfdata_path = "data/tofino2.csv"
tofino1_scale_plot_path = "plots/tofino1_scale_improvement.pdf"
tofino2_scale_plot_path = "plots/tofino2_scale_improvement.pdf"
tofino1_sw_hw_count_plot_path = "plots/tofino1_sw_hw_count.pdf"
tofino2_sw_hw_count_plot_path = "plots/tofino2_sw_hw_count.pdf"
tofino1_hw_bneck_count_plot_path = "plots/tofino1_hw_bneck_count.pdf"
tofino2_hw_bneck_count_plot_path = "plots/tofino2_hw_bneck_count.pdf"

In [ ]:
df_tofino2 = pd.read_csv(tofino2_perfdata_path, low_memory=False)
df_tofino2.head()

In [ ]:
row = df_tofino2.iloc[0]
output_string = '\n'.join([f"{col}: {value}" for col, value in row.items()])
print(output_string)

In [ ]:
sns.set()
sns.set_style("whitegrid")
font = {'family' : 'serif',
        'size'   : 25}
matplotlib.rc('font', **font)
plt.rc('xtick',labelsize=23)
plt.rc('ytick',labelsize=25)
plt.rc('axes',labelsize=30)
plt.rc('axes',titlesize=30)
plt.rc('legend',fontsize=23)
colors = list(sns.color_palette("bright"))

### Scale improvement

In [ ]:
plt.figure(figsize=(12, 7))

plt.plot(df_tofino2['num_participants'].tolist(), df_tofino2['all_min'].tolist(), color=colors[0])
plt.plot(df_tofino2['num_participants'].tolist(), df_tofino2['all_max'].tolist(), color=colors[0])
plt.fill_between(
    df_tofino2['num_participants'].tolist(), df_tofino2['all_min'].tolist(), df_tofino2['all_max'].tolist(),
    color=colors[0], alpha=0.3, label="Improvement range")

plt.xlabel("# Participants per Meeting")
plt.ylabel("Improvement vs. software")
plt.tight_layout()
plt.savefig(tofino2_scale_plot_path, dpi=300)
plt.show()

### Software and hardware ranges

In [ ]:
plt.figure(figsize=(12, 7))

plt.plot(df_tofino2['num_participants'].tolist(), df_tofino2['hw_min'].tolist(), color=colors[0])
plt.plot(df_tofino2['num_participants'].tolist(), df_tofino2['hw_max'].tolist(), color=colors[0])
plt.fill_between(
    df_tofino2['num_participants'].tolist(), df_tofino2['hw_min'].tolist(), df_tofino2['hw_max'].tolist(),
    color=colors[0], alpha=0.3, label="Scallop min/max")

plt.plot(df_tofino2['num_participants'].tolist(), df_tofino2['sw_min'].tolist(), color=colors[5])
plt.plot(df_tofino2['num_participants'].tolist(), df_tofino2['sw_max'].tolist(), color=colors[5])
plt.fill_between(
    df_tofino2['num_participants'].tolist(), df_tofino2['sw_min'].tolist(), df_tofino2['sw_max'].tolist(),
    color=colors[5], alpha=0.3, label="Software min/max")

plt.xlabel("# Participants per Meeting")
plt.ylabel("# Max. Meetings (log.)")
plt.yscale("log")
plt.legend(ncols=2)
plt.tight_layout()
plt.savefig(tofino2_sw_hw_count_plot_path, dpi=300)
plt.show()

### Software and hardware bottlenecked performance

In [ ]:
plt.figure(figsize=(12, 7))

plt.plot(df_tofino2['num_participants'].tolist(), df_tofino2['hw_bw_min'].tolist(), color=colors[3], label="Bandwidth", lw=5, drawstyle='steps-post')
plt.plot(df_tofino2['num_participants'].tolist()[1:], df_tofino2['hw_rep_min'].tolist()[1:], color=colors[0], label="Replication", lw=5, drawstyle='steps-post')
plt.plot(df_tofino2['num_participants'].tolist()[1:], df_tofino2['hw_mem_min'].tolist()[1:], color=colors[2], label="Memory", lw=5, drawstyle='steps-post')
plt.plot(df_tofino2['num_participants'].tolist(), df_tofino2['sw_min'].tolist(), color=colors[5], label="Software", lw=5, drawstyle='steps-post')
    
# Adding labels and legend
plt.rc('legend',fontsize=25)
plt.xlabel("# Participants per Meeting")
plt.ylabel("# Max. Meetings (log.)")
plt.yscale("log")
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
plt.tight_layout()
plt.savefig(tofino2_hw_bneck_count_plot_path, dpi=300)
plt.show()